In [1]:
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from catboost import Pool
import catboost as cat
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv
/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv
/kaggle/input/equity-post-HCT-survival-predictions/train.csv
/kaggle/input/equity-post-HCT-survival-predictions/test.csv
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/__script__.py
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/matplotlib-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/interface_meta-1.3.0-py3-none-any.whl
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/tzdata-2025.1-py2.py3-none-any.whl
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/pillow-11.1.0-cp310-cp310-manylinux_2_28_x86_64.whl
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/__results__.html
/kaggle/input/pm-75180884-at-02-12-2025-05-59-09/cycler-0.12.1-py3-none-any.whl
/kag

In [2]:
train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

In [3]:
train.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223


In [4]:
X = train.drop(columns=['ID', 'efs', 'efs_time'])
y = train[['efs', 'efs_time']]
X_test = test.drop(columns=['ID'])

In [5]:
num_cols = X.select_dtypes(include=[np.number]).columns
print(num_cols)
X[num_cols] = X[num_cols].fillna(X[num_cols].mean())
cat_cols = X.select_dtypes(exclude=[np.number]).columns
print(cat_cols)
for col in cat_cols:
    X[col] = X[col].fillna(X[col].mode())

Q1 = X[num_cols].quantile(0.25)
Q3 = X[num_cols].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

X_filtered = X[(X[num_cols] >= lower_bound) & (X[num_cols] <= upper_bound)]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    X[col] = le.fit_transform(X[col])


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X[num_cols] = pd.DataFrame(scaler.fit_transform(X[num_cols]), columns=num_cols)


Index(['hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6', 'hla_high_res_6',
       'hla_high_res_10', 'hla_match_dqb1_high', 'hla_nmdp_6',
       'hla_match_c_low', 'hla_match_drb1_low', 'hla_match_dqb1_low',
       'year_hct', 'hla_match_a_high', 'donor_age', 'hla_match_b_low',
       'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score', 'hla_low_res_8',
       'hla_match_drb1_high', 'hla_low_res_10'],
      dtype='object')
Index(['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',
       'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',
       'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',
       'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',
       'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',
       'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',
       'race_group', 'hepatic_mild', 'tce_div_match', 'donor

In [6]:
num_cols = X_test.select_dtypes(include=[np.number]).columns
print(num_cols)
X[num_cols] = X_test[num_cols].fillna(X_test[num_cols].median())
cat_cols = X_test.select_dtypes(exclude=[np.number]).columns
print(cat_cols)
for col in cat_cols:
    X_test[col] = X_test[col].fillna(X_test[col].mode())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    X_test[col] = le.fit_transform(X_test[col])


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_test[num_cols] = pd.DataFrame(scaler.fit_transform(X_test[num_cols]), columns=num_cols)

Index(['hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6', 'hla_high_res_6',
       'hla_high_res_10', 'hla_match_dqb1_high', 'hla_nmdp_6',
       'hla_match_c_low', 'hla_match_drb1_low', 'hla_match_dqb1_low',
       'year_hct', 'hla_match_a_high', 'donor_age', 'hla_match_b_low',
       'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score', 'hla_low_res_8',
       'hla_match_drb1_high', 'hla_low_res_10'],
      dtype='object')
Index(['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',
       'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',
       'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',
       'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',
       'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',
       'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',
       'race_group', 'hepatic_mild', 'tce_div_match', 'donor

In [7]:
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
from lifelines import KaplanMeierFitter
from sklearn.model_selection import KFold
import xgboost as xgb

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
c_index_scores = []

def transform_target(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    return y

for train_idx, val_idx in kf.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    y_train_cox = transform_target(y_train)
    y_val_cox = transform_target(y_val)

    params={
        'max_depth': 4,
        'learning_rate': 0.04898324564137106,
        'n_estimators': 403,
        'min_child_weight': 2,
        'subsample': 0.8261515350955918,
        'colsample_bytree': 0.8297150212034241,
        'reg_alpha': 0.12877762850043256,
        'reg_lambda': 0.39600640441074775
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train_cox)
    y_pred = model.predict(X_val)
    c_index = concordance_index(y_val['efs_time'], -y_pred, y_val['efs'])
    c_index_scores.append(c_index)

print(f"\nAverage C-index across {kf.get_n_splits()} folds: {np.mean(c_index_scores):.4f}")


Average C-index across 5 folds: 0.6603


In [9]:
y_pred = model.predict(X_test)
print(y_pred)
results_df = pd.DataFrame({
    'ID': test['ID'],
    'prediction': y_pred
})

print(results_df.head())
results_df.to_csv("submission.csv", index=False)

[0.5763649  0.70932543 0.63904893]
      ID  prediction
0  28800    0.576365
1  28801    0.709325
2  28802    0.639049
